In [1]:
# !pip install mlflow dagshub catboost optuna catboost

In [2]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler, PowerTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
import dagshub
import mlflow
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import optuna
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
df = pd.read_csv('/content/food_delivery_interim.csv')
df.head()

,rider_age,rider_ratings,weather,traffic_density,vehicle_condition,order_type,vehicle_type,multiple_deliveries,festival,time_taken,city_type,day_name,time_of_day,distance
0,36.0,4.2,fog,jam,2,snack,motorcycle,3.0,no,46,metropolitian,saturday,dinner_peak,10.280582
1,21.0,4.7,stormy,high,1,meal,motorcycle,1.0,no,23,metropolitian,sunday,afternoon,6.242319
2,23.0,4.7,sandstorms,medium,1,drinks,scooter,1.0,no,21,metropolitian,friday,evening_snacks,13.787860
3,34.0,4.3,sandstorms,low,0,buffet,motorcycle,0.0,no,20,metropolitian,sunday,breakfast,2.930258
4,24.0,4.7,fog,jam,1,snack,scooter,1.0,no,41,metropolitian,monday,evening_snacks,19.396618


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38055 entries, 0 to 38054
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rider_age            38055 non-null  float64
 1   rider_ratings        38055 non-null  float64
 2   weather              38055 non-null  object 
 3   traffic_density      38055 non-null  object 
 4   vehicle_condition    38055 non-null  int64  
 5   order_type           38055 non-null  object 
 6   vehicle_type         38055 non-null  object 
 7   multiple_deliveries  38055 non-null  float64
 8   festival             38055 non-null  object 
 9   time_taken           38055 non-null  int64  
 10  city_type            38055 non-null  object 
 11  day_name             38055 non-null  object 
 12  time_of_day          38055 non-null  object 
 13  distance             38055 non-null  float64
dtypes: float64(4), int64(2), object(8)
memory usage: 4.1+ MB


In [5]:
X = df.drop(columns='time_taken')
y = df['time_taken']

In [6]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
num_cols = ["rider_age","rider_ratings","distance"]

nominal_cat_cols = ["weather","order_type","vehicle_type","festival","city_type","day_name","time_of_day"]

ordinal_cat_cols = ["traffic_density"]

In [8]:
traffic_order = ["low","medium","high","jam"]

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("scale", StandardScaler(), num_cols),
        (
            "nominal_encoder",
            OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False),
            nominal_cat_cols
        ),
        (
            "ordinal_encoder",
            OrdinalEncoder(categories=[traffic_order]),
            ordinal_cat_cols
        )
    ],
    remainder="passthrough",
    n_jobs=-1,
    force_int_remainder_cols=False,
    verbose_feature_names_out=False
)

preprocessor.set_output(transform="pandas")

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('scale', StandardScaler(),
                                 ['rider_age', 'rider_ratings', 'distance']),
                                ('nominal_encoder',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['weather', 'order_type', 'vehicle_type',
                                  'festival', 'city_type', 'day_name',
                                  'time_of_day']),
                                ('ordinal_encoder',
                                 OrdinalEncoder(categories=[['low', 'medium',
                                                             'high', 'jam']]),
                                 ['traffic_density'])],
                  verbose_feature_names_out=False)

In [10]:
pt = PowerTransformer()

y_train_pt = pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt = pt.transform(y_test.values.reshape(-1,1))

In [11]:
y_train_pt = np.ravel(y_train_pt)
y_test_pt  = np.ravel(y_test_pt)

In [12]:
## pipeline

preprocessing_pipeline = Pipeline(
    steps=[
        ("preprocessor",preprocessor)
    ]
)

preprocessing_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                                   remainder='passthrough',
                                   transformers=[('scale', StandardScaler(),
                                                  ['rider_age', 'rider_ratings',
                                                   'distance']),
                                                 ('nominal_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['weather', 'order_type',
                                                   'vehicle_type', 'festival',
                                                   'city_type', 'day_name',
                                                   'time_of_day']),
                                                 ('ordinal_encoder',
                                                  OrdinalEncoder(categories=[['low',
                                                                              'medium',
                                                                              'high',
                                                                              'jam']]),
                                                  ['traffic_density'])],
                                   verbose_feature_names_out=False))])

In [13]:
X_train_trans = preprocessing_pipeline.fit_transform(X_train)
X_test_trans = preprocessing_pipeline.transform(X_test)

X_train_trans

,rider_age,rider_ratings,distance,weather_fog,weather_sandstorms,weather_stormy,weather_sunny,weather_windy,order_type_drinks,order_type_meal,...,day_name_tuesday,day_name_wednesday,time_of_day_breakfast,time_of_day_dinner_peak,time_of_day_evening_snacks,time_of_day_late_night,time_of_day_lunch_peak,traffic_density,vehicle_condition,multiple_deliveries
6965,-0.282097,1.164633,-1.211247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2,0.0
14052,1.454428,0.849370,0.717574,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0,3.0
25717,-0.455749,0.218843,0.144911,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2,1.0
35085,1.280776,-1.988001,-0.385292,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2,1.0
5921,0.759818,-0.726947,0.689886,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,-1.671317,1.164633,-1.203960,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0
6265,-0.976707,-1.672737,-0.932219,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0
11284,-0.976707,-1.672737,0.653365,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0,1.0
860,1.454428,-1.357474,0.757606,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2,2.0


In [14]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
import optuna


# ---------------- CatBoost (CPU) ----------------
best_cat = CatBoostRegressor(
    iterations=1466,
    depth=10,
    learning_rate=0.038080477733221894,
    l2_leaf_reg=26.021727151100524,
    random_strength=9.561951563676054,
    bagging_temperature=0.7315752426106197,
    loss_function="RMSE",
    eval_metric="RMSE",
    random_seed=42,
    verbose=0,
    task_type="CPU"
)


# ---------------- XGBoost (CPU) ----------------
best_xgb = XGBRegressor(
    n_estimators=309,
    max_depth=11,
    learning_rate=0.01818406589329305,
    subsample=0.8384773183505033,
    colsample_bytree=0.9244276900600374,
    reg_alpha=1.6875449963223,
    reg_lambda=6.526874838126007,
    random_state=42,
    n_jobs=-1,
    objective="reg:squarederror",
    tree_method="hist"
)


In [15]:
def objective(trial):

    meta_model_name = trial.suggest_categorical("model", ["LR", "KNN", "DT"])

    if meta_model_name == "LR":
        meta = LinearRegression()

    elif meta_model_name == "KNN":
        meta = KNeighborsRegressor(
            n_neighbors=trial.suggest_int("n_neighbors_knn", 1, 15),
            weights=trial.suggest_categorical("weights_knn", ["uniform", "distance"]),
            n_jobs=-1
        )

    else:
        meta = DecisionTreeRegressor(
            max_depth=trial.suggest_int("max_depth_dt", 1, 10),
            min_samples_split=trial.suggest_int("min_samples_split_dt", 2, 10),
            min_samples_leaf=trial.suggest_int("min_samples_leaf_dt", 1, 10),
            random_state=42
        )

    stacking_reg = StackingRegressor(
        estimators=[
            ("cat", best_cat),
            ("xgb", best_xgb)
        ],
        final_estimator=meta,
        cv=5,
        n_jobs=-1
    )

    model = TransformedTargetRegressor(
        regressor=stacking_reg,
        transformer=pt
    )

    scores = cross_val_score(
        model,
        X_train_trans,
        y_train,
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1
    )

    return -scores.mean()

In [16]:
"""
study = optuna.create_study(direction="minimize")

study.optimize(
    objective,
    n_trials=30,
    n_jobs=1,
    show_progress_bar=True
)

print("Best Params:", study.best_params)
print("Best CV MAE:", study.best_value)
"""

[I 2026-02-11 07:34:47,289] A new study created in memory with name: no-name-c36af810-abae-4582-b385-05f880e42079


  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[W 2026-02-11 07:46:16,467] Trial 0 failed with parameters: {'model': 'LR'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 206, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-2002503725.py", line 38, in objective
    scores = cross_val_score(
             ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 684, in cross_val_score
    cv_results = cross_validate(
                 ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lo

KeyboardInterrupt: 

In [ ]:
best_params = study.best_params
meta_name = best_params["model"]

if meta_name == "LR":
    final_meta = LinearRegression()

elif meta_name == "KNN":
    final_meta = KNeighborsRegressor(
        n_neighbors=best_params["n_neighbors_knn"],
        weights=best_params["weights_knn"],
        n_jobs=-1
    )

else:
    final_meta = DecisionTreeRegressor(
        max_depth=best_params["max_depth_dt"],
        min_samples_split=best_params["min_samples_split_dt"],
        min_samples_leaf=best_params["min_samples_leaf_dt"],
        random_state=42
    )


final_stacking = StackingRegressor(
    estimators=[
        ("cat", best_cat),
        ("xgb", best_xgb)
    ],
    final_estimator=final_meta,
    cv=5,
    n_jobs=-1
)

final_model = TransformedTargetRegressor(
    regressor=final_stacking,
    transformer=pt
)

final_model.fit(X_train_trans, y_train)


In [ ]:
# Predictions
y_pred_train = final_model.predict(X_train_trans)
y_pred_test = final_model.predict(X_test_trans)

# MAE
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

# R2
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

# Cross Validation
cv_scores = cross_val_score(
    final_model,
    X_train_trans,
    y_train,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

cv_mae = -cv_scores.mean()

print("\nFinal Results")
print("Train MAE:", train_mae)
print("Test MAE:", test_mae)
print("Train R2:", train_r2)
print("Test R2:", test_r2)
print("5-Fold CV MAE:", cv_mae)


In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
import optuna


# -------- CatBoost (CPU) --------
best_cat = CatBoostRegressor(
    iterations=1466,
    depth=10,
    learning_rate=0.038080477733221894,
    l2_leaf_reg=26.021727151100524,
    random_strength=9.561951563676054,
    bagging_temperature=0.7315752426106197,
    loss_function="RMSE",
    eval_metric="RMSE",
    random_seed=42,
    verbose=0,
    task_type="CPU"
)


# -------- LightGBM (CPU) --------
best_lgb = LGBMRegressor(
    n_estimators=959,
    max_depth=10,
    learning_rate=0.015908986718229125,
    num_leaves=221,
    min_child_samples=44,
    subsample=0.7916185448024788,
    colsample_bytree=0.9132622220994379,
    reg_alpha=0.08775694417084225,
    reg_lambda=4.563333600351618,
    random_state=42,
    n_jobs=-1
)


In [ ]:
def objective(trial):

    meta_model_name = trial.suggest_categorical("model", ["LR", "KNN", "DT"])

    if meta_model_name == "LR":
        meta = LinearRegression()

    elif meta_model_name == "KNN":
        meta = KNeighborsRegressor(
            n_neighbors=trial.suggest_int("n_neighbors_knn", 1, 15),
            weights=trial.suggest_categorical("weights_knn", ["uniform", "distance"]),
            n_jobs=-1
        )

    else:
        meta = DecisionTreeRegressor(
            max_depth=trial.suggest_int("max_depth_dt", 1, 10),
            min_samples_split=trial.suggest_int("min_samples_split_dt", 2, 10),
            min_samples_leaf=trial.suggest_int("min_samples_leaf_dt", 1, 10),
            random_state=42
        )

    stacking_reg = StackingRegressor(
        estimators=[
            ("cat", best_cat),
            ("lgb", best_lgb)
        ],
        final_estimator=meta,
        cv=5,
        n_jobs=-1
    )

    model = TransformedTargetRegressor(
        regressor=stacking_reg,
        transformer=pt
    )

    scores = cross_val_score(
        model,
        X_train_trans,
        y_train,
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1
    )

    return -scores.mean()


In [ ]:
best_params = study.best_params
meta_name = best_params["model"]

if meta_name == "LR":
    final_meta = LinearRegression()

elif meta_name == "KNN":
    final_meta = KNeighborsRegressor(
        n_neighbors=best_params["n_neighbors_knn"],
        weights=best_params["weights_knn"],
        n_jobs=-1
    )

else:
    final_meta = DecisionTreeRegressor(
        max_depth=best_params["max_depth_dt"],
        min_samples_split=best_params["min_samples_split_dt"],
        min_samples_leaf=best_params["min_samples_leaf_dt"],
        random_state=42
    )


final_stacking = StackingRegressor(
    estimators=[
        ("cat", best_cat),
        ("lgb", best_lgb)
    ],
    final_estimator=final_meta,
    cv=5,
    n_jobs=-1
)

final_model = TransformedTargetRegressor(
    regressor=final_stacking,
    transformer=pt
)

final_model.fit(X_train_trans, y_train)


In [ ]:
y_pred_train = final_model.predict(X_train_trans)
y_pred_test = final_model.predict(X_test_trans)

train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

cv_scores = cross_val_score(
    final_model,
    X_train_trans,
    y_train,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

cv_mae = -cv_scores.mean()

print("\nFinal Results")
print("Train MAE:", train_mae)
print("Test MAE:", test_mae)
print("Train R2:", train_r2)
print("Test R2:", test_r2)
print("5-Fold CV MAE:", cv_mae)


In [ ]:
# best parameter value

best_params = study.best_params

best_params

In [ ]:
# parameter value counts

study.trials_dataframe()["params_model"].value_counts()

In [ ]:
# mean scores for each meta estimator type

study.trials_dataframe().groupby(by="params_model")['value'].mean().sort_values()

In [ ]:
# best score

study.best_value

In [ ]:
# optimization history plot

optuna.visualization.plot_optimization_history(study)

In [ ]:
# parallel coord plot

optuna.visualization.plot_parallel_coordinate(study,params=["model"])

In [16]:
from lightgbm import LGBMRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score
import optuna

In [17]:
def objective(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1500),
        "max_depth": trial.suggest_int("max_depth", -1, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "objective": "regression_l1",   # MAE optimization
        "random_state": 42,
        "n_jobs": -1
    }

    lgb_model = LGBMRegressor(**params)

    model = TransformedTargetRegressor(
        regressor=lgb_model,
        transformer=pt
    )

    cv_score = cross_val_score(
        model,
        X_train_trans,
        y_train,
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1
    )

    return -cv_score.mean()

In [18]:
# ---------------- Run Study (50 Trials) ----------------

study = optuna.create_study(direction="minimize")

study.optimize(
    objective,
    n_trials=50,          # Changed to 50
    n_jobs=1,
    show_progress_bar=True
)

print("\nBest Params:", study.best_params)
print("Best CV MAE:", study.best_value)

[I 2026-02-11 07:57:33,317] A new study created in memory with name: no-name-097b0eba-43ab-443e-8aae-5cafc23786c1


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2026-02-11 07:58:27,699] Trial 0 finished with value: 3.257573780865625 and parameters: {'n_estimators': 993, 'max_depth': 6, 'learning_rate': 0.01158566064931126, 'num_leaves': 43, 'min_child_samples': 29, 'subsample': 0.6011277404480599, 'colsample_bytree': 0.5376907594019045, 'reg_alpha': 8.108845734807572, 'reg_lambda': 5.601153764445379}. Best is trial 0 with value: 3.257573780865625.
[I 2026-02-11 07:59:55,321] Trial 1 finished with value: 3.318772002819916 and parameters: {'n_estimators': 851, 'max_depth': 11, 'learning_rate': 0.2585901754958765, 'num_leaves': 285, 'min_child_samples': 42, 'subsample': 0.9934915248726892, 'colsample_bytree': 0.5239287368839682, 'reg_alpha': 3.1240666585880303, 'reg_lambda': 9.431073485601928}. Best is trial 0 with value: 3.257573780865625.
[I 2026-02-11 08:01:50,104] Trial 2 finished with value: 3.145084920968731 and parameters: {'n_estimators': 1467, 'max_depth': 8, 'learning_rate': 0.020026517756667247, 'num_leaves': 177, 'min_child_samples

In [19]:
best_lgb = LGBMRegressor(
    **study.best_params,
    random_state=42,
    n_jobs=-1
)

model = TransformedTargetRegressor(
    regressor=best_lgb,
    transformer=pt
)

model.fit(X_train_trans, y_train)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 362
[LightGBM] [Info] Number of data points in the train set: 30444, number of used features: 30
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

TransformedTargetRegressor(regressor=LGBMRegressor(colsample_bytree=0.8391692358860133,
                                                   learning_rate=0.02573805737449312,
                                                   max_depth=9,
                                                   min_child_samples=32,
                                                   n_estimators=1401, n_jobs=-1,
                                                   num_leaves=229,
                                                   random_state=42,
                                                   reg_alpha=1.7755636755231816,
                                                   reg_lambda=7.701684096165208,
                                                   subsample=0.6343080951775933),
                           transformer=PowerTransformer())

In [20]:
# Predictions
y_pred_train = model.predict(X_train_trans)
y_pred_test = model.predict(X_test_trans)

# Metrics
print("Training MAE:", mean_absolute_error(y_train, y_pred_train))
print("Test MAE:", mean_absolute_error(y_test, y_pred_test))
print("Training R2:", r2_score(y_train, y_pred_train))
print("Test R2:", r2_score(y_test, y_pred_test))

# Cross-validation on best model
scores = cross_val_score(
    model,
    X_train_trans,
    y_train,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

print("Final CV MAE:", -scores.mean())


Training MAE: 2.4713220333610195
Test MAE: 3.1056135561998235
Training R2: 0.8910473120935882
Test R2: 0.8292514969023069
Final CV MAE: 3.085033869143474
